In [9]:
class Infix:
    def __init__(self, function):
        self.function = function
    def __ror__(self, other):
        return Infix(lambda x, self=self, other=other: self.function(other, x))
    def __or__(self, other):
        return self.function(other)
    def __call__(self, value1, value2):
        return self.function(value1, value2)

In [43]:
from dolfinx import fem, mesh, plot
from ufl import dx, ds, grad, inner, exp, sin
from ufl import diff as D
import ufl

import numpy as np
import math
import matplotlib.pyplot as plt

from mpi4py import MPI
from dolfinx import io
import dolfinx.nls.petsc as nlspetsc
from petsc4py import PETSc

In [11]:
dot= Infix(inner)

In [28]:
domain_mesh = mesh.create_rectangle(comm=MPI.COMM_WORLD,
                                    points=((0, 0), (2, 3)),
                                    n=(10, 10),
                                    
                                    cell_type=mesh.CellType.triangle)

base_element = ufl.FiniteElement('Lagrange', domain_mesh.ufl_cell(), 1)
space = fem.FunctionSpace(domain_mesh, base_element * base_element)

In [26]:
teta = 0.5
dt = 1E-5
lam = 1E-2
m=1

In [42]:
q, v = ufl.TestFunctions(space)

next_func = fem.Function(space)
prew_func = fem.Function(space)

c, mu = ufl.split(next_func)
c0, mu0 = ufl.split(prew_func)
c = ufl.variable(c)

# Initial
next_func.x.array[:] = 0.0
next_func.sub(0).interpolate(
    lambda x: 0.63 + 0.02 *(0.5 - np.random.rand(x.shape[1]))
    )
next_func.x.scatter_forward()

f = 100 * c**2 * (1 - c)**2
dfdc = D(f, c)

mu_mid = (1 - teta) * mu0 + teta * mu

F0 = (c |dot| q) * dx - (c0 |dot| q) * dx + m * dt * (grad(mu_mid) |dot| grad(q)) * dx
F1 = (mu |dot| v) * dx -(dfdc |dot| v) * dx - lam * (grad(c) |dot| grad(v)) * dx
F = F0 + F1


In [48]:
# You can't set petsc options right now
problem = fem.petsc.NonlinearProblem(F, next_func)

solver = nlspetsc.NewtonSolver(MPI.COMM_WORLD, problem=problem)
solver.convergence_criterion = "incremental"
solver.rtol = 1e-6

ksp = solver.krylov_solver
opts = PETSc.Options()
option_prefix = ksp.getOptionsPrefix()
opts[f"{option_prefix}ksp_type"] = "preonly"
opts[f"{option_prefix}pc_type"] = "lu"
opts[f"{option_prefix}pc_factor_mat_solver_type"] = "mumps"
ksp.setFromOptions()

In [ ]:
t=0.0


In [49]:
import os